# Tanzania Regression

Generate a regression model for populations in Tanzania with/without children.

### Setup

In [1]:
!pip install -r requirements.txt

## Data

In [2]:
Tanzania_Consumption_Data = '1eRq1SbS5xTBkTzpjxLQLAKQ-j4o5UVp4mhHZSsn8K0M'
Tanzania_Family_Data = '1POplhyd44h-Zt8jMhXa2FGAzS051TPoLa2xMlmvVLU0'
Tanzania_ProcessedFood_Data = '1UaFXpfQsB-VDq4MSVt2e9WoAXqc7Rmam_-u8dvcfleo'
InputFiles = {'Expenditures':('1POplhyd44h-Zt8jMhXa2FGAzS051TPoLa2xMlmvVLU0','Expenditures'),
              'Prices':('1XaJJuVpHAJ2kK-wOg7CfdENc6XISYY_NKL0lK9wGEcU','Food Prices'),
              'HH Characteristics':('1POplhyd44h-Zt8jMhXa2FGAzS051TPoLa2xMlmvVLU0','HH Characteristics'),
              'FCT':('13cuZwMSDFtiDxUARUQc72kLj2QDv5gqPGbgjTijuUzc','For Use in Code'),
              'RDI':('1KS9270VK5mkFXalZPavxa85rHg3jiuLyL-oetUrf1m4','diet_minimums'),}

In [3]:
import pandas as pd
import numpy as np
from eep153_tools.sheets import read_sheets
import warnings
import cfe.regression as rgsn
from cfe.estimation import drop_columns_wo_covariance
warnings.filterwarnings('ignore')

pd.set_option("max_colwidth", 10)

# Pull Tazania Expenditures from Sheets
x = pd.read_pickle('Tanzania_LSMS_Expenditures.pickle')
x = x[x['m'] != 'Nan'] #Remove rows with no location
x = x.astype({'i':'int64'}) 
x = x[x.t != '2019-20']
x = x[x.t != '2020-21']


# Reformat Expedentures databases to match structure
x = x.pivot_table(values='value_purchase', index=['i','t','m'], columns=['j'], aggfunc='first')
x = x.replace(0,np.nan) # Replace zeros with missing
x = x.reset_index(level=['m'])
x

Missing dependencies for OracleDemands.


j                      m  Beef including minced sausage  Bottled beer   
i     t                                                                 
1     2008-09  Mainla...     2500.0                            NaN     \
      2010-11  Mainla...        NaN                            NaN      
      2012-13  Mainla...     2000.0                            NaN      
      2014-15  Mainla...        NaN                            NaN      
2     2008-09  Mainla...     2500.0                            NaN      
...                  ...        ...                            ...      
14980 2014-15  Mainla...     7500.0                            NaN      
14981 2014-15  Mainla...    10000.0                            NaN      
14983 2014-15  Mainla...    10000.0                            NaN      
14984 2014-15  Mainla...     1000.0                            NaN      
14985 2014-15  Mainla...     5000.0                            NaN      

j              Bread, white  Butter refined - ghee  Canned milk/milk powder   
i     t                                                                       
1     2008-09        NaN           NaN                    NaN                \
      2010-11        NaN           NaN                    NaN                 
      2012-13        NaN           NaN                    NaN                 
      2014-15        NaN           NaN                    NaN                 
2     2008-09        NaN           NaN                    NaN                 
...                  ...           ...                    ...                 
14980 2014-15        NaN           NaN                    NaN                 
14981 2014-15        NaN           NaN                    NaN                 
14983 2014-15        NaN           NaN                    NaN                 
14984 2014-15        NaN           NaN                    NaN                 
14985 2014-15        NaN           NaN                    NaN                 

j              Canned, dried and wild vegetables  Cassava Fresh, raw   
i     t                                                                
1     2008-09        NaN                                NaN           \
      2010-11        NaN                                NaN            
      2012-13        NaN                                NaN            
      2014-15        NaN                                NaN            
2     2008-09        NaN                                NaN            
...                  ...                                ...            
14980 2014-15        NaN                                NaN            
14981 2014-15        NaN                                NaN            
14983 2014-15        NaN                                NaN            
14984 2014-15        NaN                                NaN            
14985 2014-15        NaN                                NaN            

j              Cassava dry/flour  Chicken and other poultry  ...   Sugar   
i     t                                                      ...           
1     2008-09        NaN                NaN                  ...   600.0  \
      2010-11        NaN                NaN                  ...  3150.0   
      2012-13        NaN                NaN                  ...     NaN   
      2014-15        NaN                NaN                  ...  1000.0   
2     2008-09        NaN                NaN                  ...   600.0   
...                  ...                ...                  ...     ...   
14980 2014-15        NaN                NaN                  ...  2500.0   
14981 2014-15        NaN                NaN                  ...  4900.0   
14983 2014-15        NaN                NaN                  ...     NaN   
14984 2014-15        NaN                NaN                  ...     NaN   
14985 2014-15        NaN                NaN                  ...  1000.0   

j              Sugarcane   
i     t                    
1     2008-09        NaN  \
      2010-1

In [4]:
# Use to match round to year structure of Expenditures
t_map = {1: '2008-09', 2:'2010-11', 3: '2012-13', 4:'2014-15'}

# Pull additional processed food expenditures
x_processed = read_sheets(Tanzania_ProcessedFood_Data,sheet='upd4_hh_f.dta') 
x_processed = x_processed.replace({"round": t_map}) # Apply mapping
x_processed = x_processed.rename(columns={'round':'t'})
x_processed = x_processed.set_index(['i','t'])
x_processed.columns.name = 'j'
x_processed

Key available for students@eep153.iam.gserviceaccount.com.


j              Barbequed meat, chips, roast bananas  kibuku, local brews   
i     t                                                                    
1     2008-09        NaN                                   NaN            \
      2010-11        NaN                                   NaN             
      2012-13        NaN                                   NaN             
      2014-15        NaN                                   NaN             
2     2008-09        NaN                                   NaN             
...                  ...                                   ...             
45392 2014-15        NaN                                   NaN             
45393 2014-15        NaN                                   NaN             
45394 2014-15        NaN                                   NaN             
45395 2014-15        NaN                                   NaN             
45396 2014-15        NaN                                   NaN             

j              wine, commercial beer, spirits   
i     t                                         
1     2008-09        NaN                       \
      2010-11        NaN                        
      2012-13        NaN                        
      2014-15        NaN                        
2     2008-09        NaN                        
...                  ...                        
45392 2014-15        NaN                        
45393 2014-15        NaN                        
45394 2014-15        NaN                        
45395 2014-15        NaN                        
45396 2014-15        NaN                        

j              sodas and other non-alcoholic drinks  sweets, ice cream   
i     t                                                                  
1     2008-09        NaN                                   NaN          \
      2010-11        NaN                                   NaN           
      2012-13        NaN                                   NaN           
      2014-15        NaN                                   NaN           
2     2008-09        NaN                                   NaN           
...                  ...                                   ...           
45392 2014-15        NaN                                   NaN           
45393 2014-15        NaN                                   NaN           
45394 2014-15        NaN                                   NaN           
45395 2014-15        NaN                                   NaN           
45396 2014-15        NaN                                   NaN           

j              tea, coffee, samosa, cake  
i     t                                   
1     2008-09        NaN                  
      2010-11        NaN                  
      2012-13        NaN                  
      2014-15        NaN                  
2     2008-09        NaN                  
...                  ...                  
45392 2014-15        NaN                  
45393 2014-15        NaN                  
45394 2014-15        NaN                  
45395 2014-15        NaN                  
45396 2014-15        NaN                  

[83706 rows x 6 columns]

## Filter Datasets

In [5]:
# Read HH Characteristics sheet from Google,clean indexes and sets structure
hh_chrt = read_sheets(InputFiles['HH Characteristics'][0],
                    sheet=InputFiles['HH Characteristics'][1])
hh_chrt = hh_chrt.loc[hh_chrt["i"].astype(str).str.contains('-') == False]
hh_chrt["i"] = pd.to_numeric(hh_chrt["i"])
hh_chrt = hh_chrt.set_index(["i","t"])
hh_chrt

Key available for students@eep153.iam.gserviceaccount.com.


m  Females 00-03  Females 04-08  Females 09-13   
i     t                                                                 
1     2012-13  Mainla...        0.0            0.0            0.0      \
      2014-15  Mainla...        0.0            0.0            0.0       
2     2014-15  Mainla...        0.0            0.0            0.0       
4     2012-13  Mainla...        1.0            1.0            0.0       
      2014-15  Mainla...        0.0            2.0            0.0       
...                  ...        ...            ...            ...       
14972 2020-21  Mainla...        2.0            0.0            0.0       
14976 2020-21  Mainla...        0.0            0.0            0.0       
14979 2020-21  Mainla...        1.0            1.0            2.0       
14983 2020-21  Mainla...        0.0            0.0            0.0       
14985 2020-21  Mainla...        0.0            0.0            1.0       

               Females 14-18  Females 19-30  Females 31-50  Females 51-99   
i     t                                                                     
1     2012-13        0.0            0.0            0.0            1.0      \
      2014-15        0.0            0.0            0.0            1.0       
2     2014-15        0.0            1.0            0.0            0.0       
4     2012-13        0.0            1.0            0.0            0.0       
      2014-15        0.0            1.0            0.0            0.0       
...                  ...            ...            ...            ...       
14972 2020-21        1.0            1.0            1.0            0.0       
14976 2020-21        1.0            1.0            0.0            0.0       
14979 2020-21        0.0            2.0            1.0            0.0       
14983 2020-21        0.0            1.0            1.0            2.0       
14985 2020-21        1.0            0.0            0.0            0.0       

               Males 00-03  Males 04-08  Males 09-13  Males 14-18   
i     t                                                             
1     2012-13        0.0          0.0          0.0          0.0    \
      2014-15        0.0          0.0          0.0          0.0     
2     2014-15        1.0          0.0          0.0          0.0     
4     2012-13        0.0          0.0          0.0          0.0     
      2014-15        1.0          0.0          0.0          0.0     
...                  ...          ...          ...          ...     
14972 2020-21        0.0          2.0          2.0          2.0     
14976 2020-21        0.0          0.0          1.0          1.0     
14979 2020-21        2.0          2.0          1.0          1.0     
14983 2020-21        1.0          1.0          1.0          0.0     
14985 2020-21        0.0          0.0          0.0          0.0     

               Males 19-30  Males 31-50  Males 51-99  log HSize  Rural  
i     t                                                                 
1     2012-13        2.0          0.0          1.0     1.386294    1.0  
      2014-15        1.0          1.0          1.0     1.386294    1.0  
2     2014-15        1.0          0.0          0.0     1.098612    1.0  
4     2012-13        0.0          1.0          0.0     1.386294    1.0  
      2014-15        0.0          1.0          0.0     1.609438    1.0  
...                  ...          ...          ...          ...    ...  
14972 2020-21        1.0          0.0          1.0     2.564949    1.0  
14976 2020-21        1.0          0.0          1.0     1.791759    1.0  
14979 2020-21        0.0          0.0          0.0     2.564949    1.0  
14983 2020-21        1.0          0.0          0.0     2.079442    1.0  
14985 2020-21        0.0          1.0          0.0     1.098612    1.0  

[11088 rows x 17 columns]

In [6]:
# Only keep the households who have no children and merge them with their expenditures.
hh_chrt_nochildren = hh_chrt.loc[(hh_chrt['Females 00-03'] == 0) & (hh_chrt['Females 04-08'] == 0) & (hh_chrt['Females 09-13'] == 0) & 
                              (hh_chrt['Males 00-03'] == 0) & (hh_chrt['Males 04-08'] == 0) & (hh_chrt['Males 09-13'] == 0)]
hh_chrt_nochildren = hh_chrt_nochildren['Rural']
x_nochildren = x.join(hh_chrt_nochildren, how='inner')
x_nochildren.columns.name = 'j'
x_nochildren = x_nochildren.drop("Rural", axis=1)
x_nochildren

j                      m  Beef including minced sausage  Bottled beer   
i     t                                                                 
1     2012-13  Mainla...     2000.0                            NaN     \
      2014-15  Mainla...        NaN                            NaN      
14    2012-13  Mainla...        NaN                            NaN      
18    2012-13  Mainla...     2500.0                            NaN      
46    2012-13  Dar Es...     1300.0                            NaN      
...                  ...        ...                            ...      
14530 2014-15  Mainla...        NaN                            NaN      
14539 2014-15  Mainla...        NaN                            NaN      
14572 2014-15  Mainla...    10000.0                            NaN      
14755 2014-15  Mainla...        NaN                            NaN      
14776 2014-15  Mainla...    10000.0                            NaN      

j              Bread, white  Butter refined - ghee  Canned milk/milk powder   
i     t                                                                       
1     2012-13        NaN           NaN                    NaN                \
      2014-15        NaN           NaN                    NaN                 
14    2012-13        NaN           NaN                    NaN                 
18    2012-13        NaN           NaN                    NaN                 
46    2012-13        NaN           NaN                    NaN                 
...                  ...           ...                    ...                 
14530 2014-15        NaN           NaN                    NaN                 
14539 2014-15        NaN           NaN                    NaN                 
14572 2014-15     6000.0           NaN                    NaN                 
14755 2014-15        NaN           NaN                    NaN                 
14776 2014-15     3000.0           NaN                    NaN                 

j              Canned, dried and wild vegetables  Cassava Fresh, raw   
i     t                                                                
1     2012-13        NaN                                NaN           \
      2014-15        NaN                                NaN            
14    2012-13        NaN                              500.0            
18    2012-13        NaN                                NaN            
46    2012-13        NaN                             3000.0            
...                  ...                                ...            
14530 2014-15        NaN                                NaN            
14539 2014-15        NaN                                NaN            
14572 2014-15        NaN                                NaN            
14755 2014-15        NaN                                NaN            
14776 2014-15        NaN                                NaN            

j              Cassava dry/flour  Chicken and other poultry  ...   Sugar   
i     t                                                      ...           
1     2012-13        NaN                NaN                  ...     NaN  \
      2014-15        NaN                NaN                  ...  1000.0   
14    2012-13        NaN                NaN                  ...  3500.0   
18    2012-13        NaN                NaN                  ...     NaN   
46    2012-13        NaN                NaN                  ...  1000.0   
...                  ...                ...                  ...     ...   
14530 2014-15        NaN                NaN                  ...     NaN   
14539 2014-15        NaN                NaN                  ...     NaN   
14572 2014-15        NaN                NaN                  ...  1000.0   
14755 2014-15        NaN                NaN                  ...     NaN   
14776 2014-15        NaN                NaN                  ...  3000.0   

j              Sugarcane   
i     t                    
1     2012-13        NaN  \
      2014-1

In [7]:
# Only keep the households who have children and merge them with their expenditures.
hh_chrt_children = hh_chrt.loc[(hh_chrt['Females 00-03'] > 0) | (hh_chrt['Females 04-08'] > 0) | (hh_chrt['Females 09-13'] > 0) | 
                              (hh_chrt['Males 00-03'] > 0) | (hh_chrt['Males 04-08'] > 0) | (hh_chrt['Males 09-13'] > 0)]
hh_chrt_children = hh_chrt_children['Rural']
x_children = x.join(hh_chrt_children, how='inner')
x_children.columns.name = 'j'
x_children = x_children.drop("Rural", axis=1)
x_children

j                      m  Beef including minced sausage  Bottled beer   
i     t                                                                 
2     2014-15  Mainla...        NaN                            NaN     \
4     2012-13  Mainla...        NaN                            NaN      
      2014-15  Mainla...        NaN                            NaN      
5     2012-13  Mainla...     8000.0                            NaN      
      2014-15  Mainla...    15000.0                            NaN      
...                  ...        ...                            ...      
14974 2014-15  Mainla...        NaN                            NaN      
14979 2014-15  Mainla...    12000.0                            NaN      
14980 2014-15  Mainla...     7500.0                            NaN      
14981 2014-15  Mainla...    10000.0                            NaN      
14983 2014-15  Mainla...    10000.0                            NaN      

j              Bread, white  Butter refined - ghee  Canned milk/milk powder   
i     t                                                                       
2     2014-15        NaN           NaN                    NaN                \
4     2012-13        NaN           NaN                    NaN                 
      2014-15        NaN           NaN                    NaN                 
5     2012-13        NaN           NaN                    NaN                 
      2014-15     4000.0        1500.0                    NaN                 
...                  ...           ...                    ...                 
14974 2014-15        NaN           NaN                    NaN                 
14979 2014-15        NaN           NaN                    NaN                 
14980 2014-15        NaN           NaN                    NaN                 
14981 2014-15        NaN           NaN                    NaN                 
14983 2014-15        NaN           NaN                    NaN                 

j              Canned, dried and wild vegetables  Cassava Fresh, raw   
i     t                                                                
2     2014-15        NaN                                NaN           \
4     2012-13        NaN                                NaN            
      2014-15        NaN                                NaN            
5     2012-13      600.0                                NaN            
      2014-15        NaN                              500.0            
...                  ...                                ...            
14974 2014-15        NaN                                NaN            
14979 2014-15        NaN                                NaN            
14980 2014-15        NaN                                NaN            
14981 2014-15        NaN                                NaN            
14983 2014-15        NaN                                NaN            

j              Cassava dry/flour  Chicken and other poultry  ...   Sugar   
i     t                                                      ...           
2     2014-15        NaN                NaN                  ...  2000.0  \
4     2012-13        NaN                NaN                  ...   500.0   
      2014-15        NaN                NaN                  ...  3500.0   
5     2012-13        NaN                NaN                  ...  4000.0   
      2014-15        NaN                NaN                  ...  4000.0   
...                  ...                ...                  ...     ...   
14974 2014-15        NaN                NaN                  ...     NaN   
14979 2014-15        NaN                NaN                  ...  5000.0   
14980 2014-15        NaN                NaN                  ...  2500.0   
14981 2014-15        NaN                NaN                  ...  4900.0   
14983 2014-15        NaN                NaN                  ...     NaN   

j              Sugarcane   
i     t                    
2     2014-15        NaN  \
4     2012-1

In [8]:
# Merge Processed Food Expenditures with Expenditures sheet
x_children = pd.merge(x_children, x_processed, on=['i','t'], how="left")
x_children = x_children.reset_index().set_index(['i','t','m'])
x_children

j                         Beef including minced sausage  Bottled beer   
i     t       m                                                         
2     2014-15 Mainlan...        NaN                            NaN     \
4     2012-13 Mainlan...        NaN                            NaN      
      2014-15 Mainlan...        NaN                            NaN      
5     2012-13 Mainlan...     8000.0                            NaN      
      2014-15 Mainlan...    15000.0                            NaN      
...                             ...                            ...      
14974 2014-15 Mainlan...        NaN                            NaN      
14979 2014-15 Mainlan...    12000.0                            NaN      
14980 2014-15 Mainlan...     7500.0                            NaN      
14981 2014-15 Mainlan...    10000.0                            NaN      
14983 2014-15 Mainlan...    10000.0                            NaN      

j                         Bread, white  Butter refined - ghee   
i     t       m                                                 
2     2014-15 Mainlan...        NaN           NaN              \
4     2012-13 Mainlan...        NaN           NaN               
      2014-15 Mainlan...        NaN           NaN               
5     2012-13 Mainlan...        NaN           NaN               
      2014-15 Mainlan...     4000.0        1500.0               
...                             ...           ...               
14974 2014-15 Mainlan...        NaN           NaN               
14979 2014-15 Mainlan...        NaN           NaN               
14980 2014-15 Mainlan...        NaN           NaN               
14981 2014-15 Mainlan...        NaN           NaN               
14983 2014-15 Mainlan...        NaN           NaN               

j                         Canned milk/milk powder   
i     t       m                                     
2     2014-15 Mainlan...        NaN                \
4     2012-13 Mainlan...        NaN                 
      2014-15 Mainlan...        NaN                 
5     2012-13 Mainlan...        NaN                 
      2014-15 Mainlan...        NaN                 
...                             ...                 
14974 2014-15 Mainlan...        NaN                 
14979 2014-15 Mainlan...        NaN                 
14980 2014-15 Mainlan...        NaN                 
14981 2014-15 Mainlan...        NaN                 
14983 2014-15 Mainlan...        NaN                 

j                         Canned, dried and wild vegetables   
i     t       m                                               
2     2014-15 Mainlan...        NaN                          \
4     2012-13 Mainlan...        NaN                           
      2014-15 Mainlan...        NaN                           
5     2012-13 Mainlan...      600.0                           
      2014-15 Mainlan...        NaN                           
...                             ...                           
14974 2014-15 Mainlan...        NaN                           
14979 2014-15 Mainlan...        NaN                           
14980 2014-15 Mainlan...        NaN                           
14981 2014-15 Mainlan...        NaN                           
14983 2014-15 Mainlan...        NaN                           

j                         Cassava Fresh, raw  Cassava dry/flour   
i     t       m                                                   
2     2014-15 Mainlan...        NaN                 NaN          \
4     2012-13 Mainlan...        NaN                 NaN           
      2014-15 Mainlan...        NaN                 NaN           
5     2012-13 Mainlan...        NaN                 NaN           
      2014-15 Mainlan...      500.0                 NaN           
...                             ...                 ...           
14974 2014-15 Mainlan...        NaN                 NaN           
14979 2014-15 Mainlan...        NaN                 NaN           
14980 2014-15 Mainlan...

In [9]:
# Merge Processed Food Expenditures with Expenditures sheet
x_nochildren = pd.merge(x_nochildren, x_processed, on=['i','t'], how="left")
x_nochildren = x_nochildren.reset_index().set_index(['i','t','m'])
x_nochildren

j                         Beef including minced sausage  Bottled beer   
i     t       m                                                         
1     2012-13 Mainlan...     2000.0                            NaN     \
      2014-15 Mainlan...        NaN                            NaN      
14    2012-13 Mainlan...        NaN                            NaN      
18    2012-13 Mainlan...     2500.0                            NaN      
46    2012-13 Dar Es ...     1300.0                            NaN      
...                             ...                            ...      
14530 2014-15 Mainlan...        NaN                            NaN      
14539 2014-15 Mainlan...        NaN                            NaN      
14572 2014-15 Mainlan...    10000.0                            NaN      
14755 2014-15 Mainlan...        NaN                            NaN      
14776 2014-15 Mainlan...    10000.0                            NaN      

j                         Bread, white  Butter refined - ghee   
i     t       m                                                 
1     2012-13 Mainlan...        NaN           NaN              \
      2014-15 Mainlan...        NaN           NaN               
14    2012-13 Mainlan...        NaN           NaN               
18    2012-13 Mainlan...        NaN           NaN               
46    2012-13 Dar Es ...        NaN           NaN               
...                             ...           ...               
14530 2014-15 Mainlan...        NaN           NaN               
14539 2014-15 Mainlan...        NaN           NaN               
14572 2014-15 Mainlan...     6000.0           NaN               
14755 2014-15 Mainlan...        NaN           NaN               
14776 2014-15 Mainlan...     3000.0           NaN               

j                         Canned milk/milk powder   
i     t       m                                     
1     2012-13 Mainlan...        NaN                \
      2014-15 Mainlan...        NaN                 
14    2012-13 Mainlan...        NaN                 
18    2012-13 Mainlan...        NaN                 
46    2012-13 Dar Es ...        NaN                 
...                             ...                 
14530 2014-15 Mainlan...        NaN                 
14539 2014-15 Mainlan...        NaN                 
14572 2014-15 Mainlan...        NaN                 
14755 2014-15 Mainlan...        NaN                 
14776 2014-15 Mainlan...        NaN                 

j                         Canned, dried and wild vegetables   
i     t       m                                               
1     2012-13 Mainlan...        NaN                          \
      2014-15 Mainlan...        NaN                           
14    2012-13 Mainlan...        NaN                           
18    2012-13 Mainlan...        NaN                           
46    2012-13 Dar Es ...        NaN                           
...                             ...                           
14530 2014-15 Mainlan...        NaN                           
14539 2014-15 Mainlan...        NaN                           
14572 2014-15 Mainlan...        NaN                           
14755 2014-15 Mainlan...        NaN                           
14776 2014-15 Mainlan...        NaN                           

j                         Cassava Fresh, raw  Cassava dry/flour   
i     t       m                                                   
1     2012-13 Mainlan...        NaN                 NaN          \
      2014-15 Mainlan...        NaN                 NaN           
14    2012-13 Mainlan...      500.0                 NaN           
18    2012-13 Mainlan...        NaN                 NaN           
46    2012-13 Dar Es ...     3000.0                 NaN           
...                             ...                 ...           
14530 2014-15 Mainlan...        NaN                 NaN           
14539 2014-15 Mainlan...        NaN                 NaN           
14572 2014-15 Mainlan...

In [10]:
 def gen_regression(x):
    # Take logs of expenditures; call this y
    y = np.log(x.replace(0,np.nan).dropna(how='all'))

    # Change 'Tanzania' to key of your own sheet in Sheets, above
    d = read_sheets(Tanzania_Family_Data,sheet="HH Characteristics")
    d.columns.name = 'k'
    d = d[d['m'] != 'Nan']
    d = d.astype({'i':'int64'})

    d = d.drop("Rural",axis = 1)
    d.set_index(['i','t','m'],inplace=True)

    y = drop_columns_wo_covariance(y,min_obs=17)
    use = y.index.intersection(d.index)

    y = y.loc[use,:]

    d = d.loc[use,:]

    y = y.stack()

    d = d.stack()

    # Remove duplicate indexes
    idx = np.unique(d.index.values, return_index = True)[1]
    d = d.iloc[idx] 

    # Check that indices are in right places!
    assert y.index.names == ['i','t','m','j']
    assert d.index.names == ['i','t','m','k']

    result = cfe.Regression(y=y,d=d,min_obs=17,min_prop_items=0,alltm=False)

    result.predicted_expenditures()
    return result

In [11]:
import cfe
r_children = gen_regression(x_children)
r_children.to_pickle('Tanzania_results_children.pickle')

Key available for students@eep153.iam.gserviceaccount.com.


In [12]:
r_nochildren = gen_regression(x_nochildren)
r_nochildren.to_pickle('Tanzania_results_nochildren.pickle')

Key available for students@eep153.iam.gserviceaccount.com.
